### Dataset 与 DataLoader 读取数据

Dataset类表示数据集，通过继承Dataset自定义数据集格式、大小和属性，以供后面的DataLoader使用。

#### Dataset类
继承此类需要重写:
- \_\_init\_\_() 构造函数 可以自定义数据读取方法和数据预处理
- \_\_len\_\_()  返回数据集大小
- \_\_getitem\_\_() 索引数据集中某一个数据

In [22]:
import torch
from torch.utils.data import Dataset

# Tensor类型的数据集
class MyDateset(Dataset):
    # 构造函数
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
    # 返回数据集大小
    def __len__(self):
        return self.data_tensor.size(0)
    # 返回索引的数据与标签
    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

In [23]:
# 生成数据
data_tensor = torch.randn(10, 3)    # randn 均值为0 方差为1 的标准正态分布
target_tensor = torch.randint(2, (10,))
data_tensor, target_tensor

(tensor([[-0.7899,  0.2831,  0.4905],
         [ 0.4736,  0.6144,  0.4065],
         [-0.8576,  0.0502, -1.1867],
         [-0.7946,  0.4233,  0.4343],
         [ 1.5250, -0.7599,  1.0228],
         [ 0.7295,  0.3408,  0.8373],
         [-0.1250,  0.3268, -2.1309],
         [-1.6104, -0.8479, -1.9267],
         [ 0.3549, -0.0297,  0.6682],
         [-0.7354, -1.3689,  1.9006]]),
 tensor([0, 0, 0, 0, 1, 0, 0, 0, 1, 0]))

In [24]:
# 将数据封装为Dataset
my_dataset = MyDateset(data_tensor=data_tensor, target_tensor=target_tensor)
print('Dataset size:', len(my_dataset))
print('tensor_data[0]:', my_dataset[0])

Dataset size: 10
tensor_data[0]: (tensor([-0.7899,  0.2831,  0.4905]), tensor(0))


#### DataLoader 类
如果数据量很大，考虑内存、IO问题，训练过程中不会将数据一次性全部加载到内存中，也可能用多个进程去加载，所以需要多进程，迭代加载。
DataLoader就是根据以上需求设计的。

DataLoader 是一个迭代器，最基本的使用方法就是传入一个 Dataset 对象，它会根据参数 batch_size 的值生成一个 batch 的数据，节省内存的同时，它还可以实现多进程、数据打乱等处理。

In [25]:
from torch.utils.data import DataLoader

tensor_dataloader = DataLoader(
    dataset=my_dataset,     # 传入的数据集
    batch_size=2,           # 输出的 batch 的大小
    shuffle=True,           # 是否打乱
    num_workers=0           # 进程数 0 表示只有主进程
    )

for data, target in tensor_dataloader:
    print(data, target)

# 输出一个 batch
print('One batch tensor data: ', iter(tensor_dataloader).next())

tensor([[-0.8576,  0.0502, -1.1867],
        [ 0.4736,  0.6144,  0.4065]]) tensor([0, 0])
tensor([[-0.7946,  0.4233,  0.4343],
        [-0.7899,  0.2831,  0.4905]]) tensor([0, 0])
tensor([[ 1.5250, -0.7599,  1.0228],
        [ 0.3549, -0.0297,  0.6682]]) tensor([1, 1])
tensor([[-0.1250,  0.3268, -2.1309],
        [ 0.7295,  0.3408,  0.8373]]) tensor([0, 0])
tensor([[-0.7354, -1.3689,  1.9006],
        [-1.6104, -0.8479, -1.9267]]) tensor([0, 0])
One batch tensor data:  [tensor([[-0.7354, -1.3689,  1.9006],
        [-1.6104, -0.8479, -1.9267]]), tensor([0, 0])]


In [26]:
import torchvision
from torchvision import transforms
from torchvision import datasets
# 定义一个transform
my_transform = transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize((0.5), (0.5))
                                  ])
mnist_dataset = torchvision.datasets.MNIST(
    root='D:\Coding\Dataset\MNIST',
    train=True,
    transform=my_transform,
    target_transform=None, 
    download=True)

In [27]:
mnist_dataset_list = list(mnist_dataset)
print(mnist_dataset_list)

In [ ]:
display(mnist_dataset_list[0][0])
print("Image label is:", mnist_dataset_list[0][1])

Image label is: 5


In [ ]:
# 查看变换后的数据类型
item = mnist_dataset.__getitem__(0)
print(type(item[0]))